In [21]:
import pygame
import math
import random
pygame.init()

width, height = 600, 400
screen = pygame.display.set_mode((600, 400))
pygame.display.set_caption("Проект")

background = pygame.image.load("back.jpg")
background = pygame.transform.scale(background, (600, 400))

cat_image = pygame.image.load("cat.jpg").convert()
cat_image = pygame.transform.scale(cat_image, (width//6, height//4))
cat_image.set_colorkey((255, 255, 255))

hoop_width, hoop_height = 60, 60
mouse_image = pygame.image.load("mouse.jpg").convert()
mouse_image = pygame.transform.scale(mouse_image, (hoop_width,hoop_height))
mouse_image.set_colorkey((255, 255, 255))

pygame.mixer.init()
pygame.mixer.music.load("silent-embrace_90136.wav")
pygame.mixer.music.play(-1) 

score_sound = pygame.mixer.Sound("cat-meows.wav")  

start_x1, start_y1 = 100, 300
step1 = 30
x1, y1 = start_x1, start_y1
radius = 30

vx1, vy1 = 0, 0
gravity = 0.5
power = 12
angle = 45
bounce_loss = 0.7
on_ground = True

hoop_x1 = random.randint(60, 600 - 60)
hoop_y1 = random.randint(60, 400 - 60)

font = pygame.font.Font(None, 24)

scored_flag = False
score = 0

x, y = 550, 300
speed = 0.65
running = True

class MovingSquare:
    def __init__(self, screen, x, y, speed):
        self.screen = screen
        self.x = x
        self.y = y
        self.speed = speed

    def update(self):
        self.x -= self.speed
        if self.x < 50:
            self.x = 600

    def draw(self):
        pygame.draw.rect(self.screen, (255, 0, 0), (self.x, self.y, 10, 10))

    def red_rect(self):
        return pygame.Rect(self.x, self.y, 10, 10)
square = MovingSquare(screen, x, y, speed)
 
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                angle = min(angle + 5, 80)
            elif event.key == pygame.K_RIGHT:
                angle = max(angle - 5, 10)
            elif event.key == pygame.K_w:
                power = min(power + 1, 25)
            elif event.key == pygame.K_s:
                power = max(power - 1, 5)
            elif event.key == pygame.K_SPACE and on_ground:
                rad = math.radians(angle)
                vx1 = power * math.cos(rad)
                vy1 = -power * math.sin(rad)
                on_ground = False    
    
    if not on_ground:
        vy1 += gravity
        x1 += vx1
        y1 += vy1

    if y1 + radius >= 400:
        y1 = 400 - radius
        vy1 = -vy1 * bounce_loss
        vx1 *= bounce_loss
        if abs(vy1) < 1 and abs(vx1) < 1:
            x1, y1 = start_x1, start_y1
            vx1, vy1 = 0, 0
            on_ground = True
            scored_flag = False
            hoop_x1 = random.randint(60, 600 - 60)
            hoop_y1 = random.randint(60, 400 - 60)
    
    hoop_rect = pygame.Rect(hoop_x1, hoop_y1, hoop_width, hoop_height)
    cat_rect = pygame.Rect(x1 - radius, y1 - radius, radius * 2, radius * 2)
    red_rect_t = square.red_rect()
    
    if cat_rect.colliderect(red_rect_t):
        screen.fill((0, 0, 0))
        break
        
    if cat_rect.colliderect(hoop_rect) and not scored_flag:
        scored_flag = True
        score += 1
        score_sound.play()
    
    screen.blit(background, (0, 0))

    if on_ground:
        points = []
        rad = math.radians(angle)
        temp_vx1 = power * math.cos(rad)
        temp_vy1 = -power * math.sin(rad)
        temp_x1, temp_y1 = start_x1, start_y1
        for i in range(60):
            temp_vy1 += gravity
            temp_x1 += temp_vx1
            temp_y1 += temp_vy1
            if temp_y1 + radius >= 400:
                break
            points.append((int(temp_x1), int(temp_y1)))
        if len(points) > 1:
            pygame.draw.lines(screen, (0, 0, 0), False, points, 2)
    
    screen.blit(cat_image, (int(x1 - radius), int(y1 - radius)))
    screen.blit(mouse_image, (hoop_x1, hoop_y1))

    text_angle = font.render(f"Угол: {angle}°", True, (255, 0, 0))
    text_power = font.render(f"Сила: {power}", True, (255, 0, 0))
    text_score = font.render(f"Счёт: {score}", True, (255, 0, 0))
    screen.blit(text_angle, (10, 10))
    screen.blit(text_power, (10, 30))
    screen.blit(text_score, (10, 50))

    if scored_flag:
        hit_text = font.render("ПОПАДАНИЕ!", True, (0, 255, 0))
        screen.blit(hit_text, (250, 50))
    
    square.update()
   
    square.draw()
    
    pygame.display.flip()
    pygame.time.delay(20)

pygame.quit()